In [146]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress

# Import API key
from api_keys import weather_api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

In [147]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

617

In [148]:
#Make a request for each of the cities
url = "http://api.openweathermap.org/data/2.5/weather?"
query_url = f"{url}appid={weather_api_key}&q="

import json

responses = []

for x in range(len(cities)):
    print(f"Processing Record {x} of Set 1 | {cities[x]}")
    
    response = requests.get(query_url + cities[x])
    responses.append(response.json())

Processing Record 0 of Set 1 | magdalena
Processing Record 1 of Set 1 | bambous virieux
Processing Record 2 of Set 1 | ushuaia
Processing Record 3 of Set 1 | pecos
Processing Record 4 of Set 1 | olinda
Processing Record 5 of Set 1 | hermanus
Processing Record 6 of Set 1 | cidreira
Processing Record 7 of Set 1 | chokurdakh
Processing Record 8 of Set 1 | kemise
Processing Record 9 of Set 1 | cape town
Processing Record 10 of Set 1 | butaritari
Processing Record 11 of Set 1 | vaini
Processing Record 12 of Set 1 | roebourne
Processing Record 13 of Set 1 | san patricio
Processing Record 14 of Set 1 | berlevag
Processing Record 15 of Set 1 | punta arenas
Processing Record 16 of Set 1 | catuday
Processing Record 17 of Set 1 | rikitea
Processing Record 18 of Set 1 | cabo san lucas
Processing Record 19 of Set 1 | muros
Processing Record 20 of Set 1 | busselton
Processing Record 21 of Set 1 | bluff
Processing Record 22 of Set 1 | rocha
Processing Record 23 of Set 1 | atuona
Processing Record 24 

In [151]:
with open('output_data/responses.json', 'w') as json_file:
    json.dump(responses, json_file)

In [153]:
df = pd.read_json (r'output_data/responses.json')
df.to_csv (r'output_data/cities.csv', index = None)

In [149]:
responses_df = pd.DataFrame(responses)
responses_df

,coord,weather,base,main,visibility,wind,clouds,dt,sys,timezone,id,name,cod,rain,message,snow
0,"{'lon': 121.429, 'lat': 14.1999}","[{'id': 803, 'main': 'Clouds', 'description': ...",stations,"{'temp': 299.54, 'feels_like': 303.19, 'temp_m...",10000.0,"{'speed': 1.03, 'deg': 0}",{'all': 75},1.610118e+09,"{'type': 1, 'id': 8160, 'country': 'PH', 'sunr...",28800.0,1703912.0,Magdalena,200,NaN,NaN,NaN
1,"{'lon': 57.7575, 'lat': -20.3428}","[{'id': 500, 'main': 'Rain', 'description': 'l...",stations,"{'temp': 299.73, 'feels_like': 299.51, 'temp_m...",8000.0,"{'speed': 6.69, 'deg': 120}",{'all': 75},1.610118e+09,"{'type': 1, 'id': 2061, 'country': 'MU', 'sunr...",14400.0,1106677.0,Bambous Virieux,200,{'1h': 0.21},NaN,NaN
2,"{'lon': -68.3, 'lat': -54.8}","[{'id': 500, 'main': 'Rain', 'description': 'l...",stations,"{'temp': 281.15, 'feels_like': 271.56, 'temp_m...",10000.0,"{'speed': 11.83, 'deg': 230}",{'all': 75},1.610118e+09,"{'type': 1, 'id': 8303, 'country': 'AR', 'sunr...",-10800.0,3833367.0,Ushuaia,200,{'1h': 0.3},NaN,NaN
3,"{'lon': -103.4932, 'lat': 31.4229}","[{'id': 800, 'main': 'Clear', 'description': '...",stations,"{'temp': 272.15, 'feels_like': 267.74, 'temp_m...",10000.0,"{'speed': 2.57, 'deg': 340}",{'all': 1},1.610118e+09,"{'type': 1, 'id': 5325, 'country': 'US', 'sunr...",-21600.0,5528182.0,Pecos,200,NaN,NaN,NaN
4,"{'lon': -34.8553, 'lat': -8.0089}","[{'id': 802, 'main': 'Clouds', 'description': ...",stations,"{'temp': 301.15, 'feels_like': 301.14, 'temp_m...",10000.0,"{'speed': 5.14, 'deg': 100}",{'all': 40},1.610118e+09,"{'type': 1, 'id': 8426, 'country': 'BR', 'sunr...",-10800.0,3393536.0,Olinda,200,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
612,"{'lon': -53.0572, 'lat': -25.7336}","[{'id': 800, 'main': 'Clear', 'description': '...",stations,"{'temp': 302.08, 'feels_like': 303.16, 'temp_m...",10000.0,"{'speed': 2.12, 'deg': 170}",{'all': 1},1.610118e+09,"{'country': 'BR', 'sunrise': 1610095764, 'suns...",-10800.0,3464579.0,Dois Vizinhos,200,NaN,NaN,NaN
613,"{'lon': -60.4388, 'lat': -26.7852}","[{'id': 800, 'main': 'Clear', 'description': '...",stations,"{'temp': 307.59, 'feels_like': 310.18, 'temp_m...",10000.0,"{'speed': 1.34, 'deg': 315, 'gust': 6.71}",{'all': 1},1.610118e+09,"{'type': 3, 'id': 2033293, 'country': 'AR', 's...",-10800.0,3840300.0,Presidencia Roque Sáenz Peña,200,NaN,NaN,NaN
614,"{'lon': 62.3497, 'lat': 59.4307}","[{'id': 804, 'main': 'Clouds', 'description': ...",stations,"{'temp': 256.17, 'feels_like': 252.34, 'temp_m...",2115.0,"{'speed': 0.47, 'deg': 25}",{'all': 99},1.610118e+09,"{'country': 'RU', 'sunrise': 1610080960, 'suns...",18000.0,1506676.0,Gari,200,NaN,NaN,NaN
615,"{'lon': 11.0694, 'lat': 11.7091}","[{'id': 803, 'main': 'Clouds', 'description': ...",stations,"{'temp': 308.8, 'feels_like': 301.21, 'temp_mi...",10000.0,"{'speed': 7.87, 'deg': 62}",{'all': 71},1.610118e+09,"{'country': 'NG', 'sunrise': 1610084279, 'suns...",3600.0,2324767.0,Potiskum,200,NaN,NaN,NaN


In [ ]:
df = pd.DataFrame({
    city = response_json['sys']['name']
    lat = response_json['coord']['lat']
    lng = response_json['coord']['lon']
    max_temp = response_json['main']['temp_max']
    humidity = response_json['main']['humidity']
    cloudiness = response_json['clouds']['all']
    wind_speed = response_json['wind']['speed']
    country = response_json['sys']['country']
    date = response_json['dt']
})

In [75]:
import csv
with open(output_data_file,'w') as csvfile:
    csvwriter = csv.writer(output_data_file, delimiter = ",")

TypeError: argument 1 must have a "write" method